In [3]:
import time
import numpy as np
import tweepy
import os
from qiskit import QuantumCircuit, Aer, transpile, execute
from qiskit import Aer
from qiskit import IBMQ
provider = IBMQ.load_account()# from qiskit.visualization import plot_histogram, plot_bloch_multivector

from quantuminspire.credentials import get_authentication
from quantuminspire.qiskit import QI
QI_URL = os.getenv('QI_URL', 'https://api.quantum-inspire.com/')

from classical_channel import authenticate, USERS

In [4]:
QM = USERS['QM']
ALICE = USERS['RK']
BOB = USERS['AG']

In [5]:
INTERFACE, START = authenticate(QM)
START

Authentication OK


1643552254161

In [6]:
def get_gates(auth_alice, auth_bob):
    count = 0
    NEW = int(INTERFACE.get_direct_messages()[0].created_timestamp)
    while START >= NEW:
        time.sleep(60)
        count += 1
        NEW = int(INTERFACE.get_direct_messages()[0].created_timestamp)
        print(count,'  start',START,'  new', NEW)
        if count >= 6:
            print('TIME OUT')
            NEW = START + 10
            break
    direct_msgs = INTERFACE.get_direct_messages()
    Alice_msgs = list(filter(lambda x: (x.message_create['sender_id'] == auth_alice), direct_msgs))
    Bob_msgs = list(filter(lambda x: (x.message_create['sender_id'] == auth_bob), direct_msgs))
    Alice_gate = Alice_msgs[0].message_create['message_data']['text']
    Bob_gate = Bob_msgs[0].message_create['message_data']['text']
    return Alice_gate, Bob_gate

In [7]:
alice_gate, bob_gate = get_gates(ALICE, BOB)
print(alice_gate)
print(bob_gate)

XXXZXZZZZX
XZZZXZXZXX


In [10]:
# BACKEND = Aer.get_backend("qasm_simulator")
# backend = provider.get_backend('ibmq_armonk')
authentication = get_authentication()
QI.set_authentication(authentication, QI_URL)
QI.set_project_name('QKD tweet dry run')

In [13]:
BACKEND = QI.get_backend('Starmon-5')

In [14]:
N = len(alice_gate) == len(bob_gate) and len(alice_gate)
N

10

In [15]:
def create_pairs(n):
    message = []
    for idx in range(n):
        qc = QuantumCircuit(2,2)
        qc.h(0)
        qc.cx(0,1)
        message.append(qc)
    return message

In [16]:
cirquits = create_pairs(N)

In [17]:
def apply_gates(bell_cirq, alice_gate, bob_gate):
    message = bell_cirq.copy()
    n = len(message)
    for idx in range(n):
        if alice_gate[idx] == 'Z':
            pass
        else:
            message[idx].h(0)
            
        if bob_gate[idx] == 'Z':
            pass
        else:
            message[idx].h(1)
            
            
        message[idx].measure(0,0)
        message[idx].measure(1,1)
    return message

In [18]:
MESSAGE = apply_gates(cirquits, alice_gate, bob_gate)

In [19]:
def run_circuit(message):
    message_tp = transpile(message, backend=BACKEND)
    job = execute(message_tp, backend= BACKEND, shots=1)
    result = job.result().get_counts()
    measurements = [list(result_vals.keys())[0] for result_vals in result]
    return np.array(measurements)

In [20]:
RESULTS = run_circuit(MESSAGE)

In [21]:
RESULTS

array(['00', '00', '11', '00', '11', '01', '11', '11', '01', '10'],
      dtype='<U2')

In [22]:
def send_results(auth_alice, auth_bob, measurements, alice_gate, bob_gate):
    alice_results = []
    bob_results = []
    good_bits = []
    for idx, results in enumerate(measurements):
        alice_results.append(results[0])
        bob_results.append(results[1])
        if alice_gate[idx] == bob_gate[idx]:
            good_bits.append(results[0])
    INTERFACE.send_direct_message(auth_alice, f'Results = {alice_results}')
    INTERFACE.send_direct_message(auth_bob, f'Results = {bob_results}')
    INTERFACE.send_direct_message(auth_alice, f'Initiate basis sharing?')
    out = INTERFACE.send_direct_message(auth_bob, f'Initiate basis sharing?')
    
    return good_bits, int(out.created_timestamp)

In [24]:
THE_KEY, RECIVED = send_results(ALICE, BOB, RESULTS, alice_gate, bob_gate)

In [26]:
def share_basis(auth_alice, auth_bob, alice_gate, bob_gate, key):
    count = 0
    NEW = int(INTERFACE.get_direct_messages()[0].created_timestamp)
    while RECIVED >= NEW:
        time.sleep(60)
        count += 1
        NEW = int(INTERFACE.get_direct_messages()[0].created_timestamp)
        print(count,'  start',START,'  new', NEW)
        if count >= 6:
            print('TIME OUT')
            NEW = START + 10
            break
    
    direct_msgs = INTERFACE.get_direct_messages()
    
    Alice_msgs = list(filter(lambda x: (x.message_create['sender_id'] == auth_alice), direct_msgs))
    Bob_msgs = list(filter(lambda x: (x.message_create['sender_id'] == auth_bob), direct_msgs))
    Alice_procede = Alice_msgs[0].message_create['message_data']['text']
    Bob_procede = Bob_msgs[0].message_create['message_data']['text']
    if Alice_procede==Bob_procede=='YES':
        INTERFACE.send_direct_message(auth_alice, f'Bob basis = {bob_gate} \n KEY = {key}')
        INTERFACE.send_direct_message(auth_bob, f'Alice basis = {alice_gate} \n KEY = {key}')
        return True
    else:
        print('Basis share rejected')
        return False